#### Project CCF pySpark RDD

In [2]:
from itertools import islice
g=sc.textFile("FileStore/tables/webGoogleS.txt")
#remove header 4 lines
data= g.mapPartitionsWithIndex(lambda idx, it: islice(it, 4, None) if idx == 0 else it)
data.count()

Out[1]: 41306

In [3]:
pair = data.map(lambda x: x.split("\t")).map(lambda x: (x[0], x[1]))
pair.take(5)

Out[2]: [('0', '11342'),
 ('0', '824020'),
 ('0', '867923'),
 ('0', '891835'),
 ('11342', '0')]

In [1]:
def CCF_dedup(data):
  dedup = data.map(lambda x: ((x[0],x[1]), 1))\
              .reduceByKey(lambda x,y: 1)\
              .map(lambda x: (x[0][0], x[0][1]))
  return dedup

def CCF_Iterate_map(pair):
    #map
    pair2=pair.map(lambda x: (x[1], x[0]))
    map_pair=pair.union(pair2)
    return map_pair

def f(x): return x
    
def CCF_Iterate_reduce(data):
    #find min value per key
    #couple (key, min)
    key_min=data.reduceByKey(lambda x,y: min(x,y)).filter(lambda x: x[0]>x[1])
    #put together: in one command
    valuelist_min=key_min.join(data).map(lambda x: (x[0],x[1][1])).cogroup(key_min).map(lambda x :(x[0], ( list(x[1][0]), list(x[1][1]))))
    
    #make RDD : (min, (list of value))
    min_valuelist=valuelist_min.map(lambda x:(x[1][1][0], x[1][0]))

    #use flatmapvalue to transform to RDD (min, value), then filter min != value, map to couple (value, min)

    min_value=min_valuelist.flatMapValues(f).filter(lambda x: x[0]!=x[1]).map(lambda x: (x[1],x[0]))
    countnewpair=min_value.count()

    #union couple (key, min) and (value, min)
    unionkeyminvalue=key_min.union(min_value)

    return unionkeyminvalue
  

In [2]:
r=sc.parallelize([("A","B"),("B","C"),("B","D"),("D","E"),("F","G"),("G","H")])
r.collect()

NameError: name 'sc' is not defined

In [6]:
map1= CCF_Iterate_map(r)
map1.collect()

Out[9]: [('A', 'B'),
 ('B', 'C'),
 ('B', 'D'),
 ('B', 'A'),
 ('C', 'B'),
 ('D', 'E'),
 ('D', 'B'),
 ('E', 'D'),
 ('F', 'G'),
 ('G', 'H'),
 ('G', 'F'),
 ('H', 'G')]

In [7]:
reduce1=CCF_Iterate_reduce(map1)
reduce1.collect()

Out[8]: [('B', 'A'),
 ('C', 'B'),
 ('C', 'A'),
 ('D', 'B'),
 ('D', 'A'),
 ('E', 'D'),
 ('E', 'B'),
 ('G', 'F'),
 ('H', 'G'),
 ('H', 'F')]

In [8]:
key_min=map1.reduceByKey(lambda x,y: min(x,y)).filter(lambda x: x[0]>x[1])
key_min.collect()




Out[7]: [('B', 'A'), ('D', 'B'), ('G', 'F'), ('E', 'D'), ('H', 'G'), ('C', 'B')]

In [9]:
valuelist_min=key_min.join(map1).map(lambda x: (x[0],x[1][1])).cogroup(key_min).map(lambda x :(x[0], ( list(x[1][0]), list(x[1][1]))))
valuelist_min.collect()

Out[37]: [('G', (['H', 'F'], ['F'])),
 ('E', (['D'], ['D'])),
 ('H', (['G'], ['G'])),
 ('B', (['C', 'D', 'A'], ['A'])),
 ('D', (['E', 'B'], ['B'])),
 ('C', (['B'], ['B']))]

In [10]:
min_valuelist=valuelist_min.map(lambda x:(x[1][1][0], x[1][0]))
#min_valuelist.take(10)
#use flatmapvalue to transform to RDD (min, value), then filter min != value, map to couple (value, min)
def f(x): return x
min_value=min_valuelist.flatMapValues(f).filter(lambda x: x[0]!=x[1]).map(lambda x: (x[1],x[0]))
min_value.collect()

Out[38]: [('H', 'F'), ('C', 'A'), ('D', 'A'), ('E', 'B')]

In [11]:
unionkeyminvalue=key_min.union(min_value)
unionkeyminvalue.collect()

Out[39]: [('B', 'A'),
 ('D', 'B'),
 ('G', 'F'),
 ('E', 'D'),
 ('H', 'G'),
 ('C', 'B'),
 ('H', 'F'),
 ('C', 'A'),
 ('D', 'A'),
 ('E', 'B')]